## This code loads the dataset ##

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda114]
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 1.6 MB/s            


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# necessary installations
!python3 -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl (13.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms


In [4]:

# We want to convert text -> numerical values
# 1. We need a Vocabulary mapping each word to a index
# 2. We need to setup a Pytorch dataset to load the data
# 3. Setup padding of every batch (all examples should be
#    of same seq_len and setup dataloader)
# Note that loading the image is very easy compared to the text!

# Download with: python -m spacy download en
spacy_eng = spacy.load("en_core_web_sm")


class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]




In [5]:
def get_captions(captions_file):
    data = pd.read_excel(captions_file, sheet_name=None)
    data_affinis = data['Affinis'] # needs to be update for the rest of the species
    data_affinis.rename(columns={'ID':'Image-ID', 'Anterior Scutum':'Anterior-Scutum', 'Inner-Alar Space':'Inner-Alar-Space'}, inplace=True)
    data_affinis.set_index('Image-ID', inplace=True)
    data_affinis.dropna(how='all', inplace=True)
    #data_affinis['Corbicular'].fillna("False", inplace=True)
    data_affinis.fillna("False", inplace=True)
    data_affinis['Corbicular'] = data_affinis['Corbicular'].str.strip().str.lower().str.replace("yes", "True")
    data_affinis['Corbicular'] = data_affinis['Corbicular'].str.strip().str.lower().str.replace("no", "False")
    
    for column in data_affinis.columns:
        data_affinis[f"{column}"] = f"{column}: " + data_affinis[f"{column}"]
    
    
    
    bumble_bees_attribute_description = ''
    ind = 0
    for column in data_affinis.columns:
      if ind != len(data_affinis.columns) - 1:
        bumble_bees_attribute_description += data_affinis[f"{column}"] + ', '
      else:
        bumble_bees_attribute_description += data_affinis[f"{column}"]
      ind += 1
    print(bumble_bees_attribute_description.tail())
    
    return bumble_bees_attribute_description.reset_index()

In [6]:
class BeeDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = get_captions(captions_file)
        self.transform = transform

        # Get img, caption columns
        self.imgs = self.df['Image-ID']
        self.captions = self.df['Head']

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):  
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, str(img_id)+'.jpg')).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)




In [7]:
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets


In [8]:
def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
):
    dataset = BeeDataset(root_folder, annotation_file, transform=transform)

    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader, dataset


In [9]:
transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),]
    )



In [10]:
loader, dataset = get_loader(
        "../dataset_3/training/Bombus_affinis", "../Beemachine_Experiments_Logs.xlsx", transform=transform
    )

    

Image-ID
246    Head: B, Anterior-Scutum: Y, Inner-Alar-Space:...
247    Head: B, Anterior-Scutum: Y, Inner-Alar-Space:...
248    Head: B, Anterior-Scutum: Y, Inner-Alar-Space:...
249    Head: B, Anterior-Scutum: Y, Inner-Alar-Space:...
250    Head: B, Anterior-Scutum: Y, Inner-Alar-Space:...
Name: Head, dtype: object


In [11]:
for idx, (imgs, captions) in enumerate(loader):
    print(imgs.shape)
    print(captions.shape)

torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
torch.Size([26, 3, 224, 224])
torch.Size([53, 26])
